In [ ]:
import pandas as pd
import numpy as np
import sklearn 
data = pd.read_csv("system-data.csv")

print("🔍 Checking Missing Values:")
print(data.isnull().sum())

print("\n📌 Sample Data:")
print(data.head())

🔍 Checking Missing Values:
course_id                 0
userid_DI                 0
registered                0
viewed                    0
explored                  0
certified                 0
final_cc_cname_DI         0
LoE_DI                44119
YoB                   38504
gender                32961
grade                 25890
start_time_DI             0
last_event_DI        152231
nevents              159278
ndays_act            142510
nplay_video          304946
nchapters            144465
nforum_posts              0
roles                338223
incomplete_flag      260838
dtype: int64

📌 Sample Data:
                    course_id       userid_DI  registered  viewed  explored  \
0  HarvardX/CB22x/2013_Spring  MHxPC130442623           1       0         0   
1  HarvardX/CB22x/2013_Spring  MHxPC130275857           1       0         0   
2  HarvardX/CB22x/2013_Spring  MHxPC130539455           1       1         0   
3  HarvardX/CB22x/2013_Spring  MHxPC130088379           1       1   

In [7]:
user_data = data[["userid_DI", "gender", "LoE_DI", "YoB"]].drop_duplicates()

In [8]:
course_field_mapping = {
    "CB22x": "Humanities",
    "ER22x": "Humanities",
    "CS50x": "Computer Science",
    "PH207x": "Health",
    "PH278x": "Health",
}

# Extract the course code and map to a field of study
data["course_code"] = data["course_id"].apply(lambda x: x.split("/")[1])  # Extract course code
data["field_of_study"] = data["course_code"].map(course_field_mapping)


In [9]:
# Fill missing values correctly
data.loc[:, "LoE_DI"] = data["LoE_DI"].fillna(data["LoE_DI"].mode()[0])
data.loc[:, "YoB"] = data["YoB"].fillna(data["YoB"].median())
data.loc[:, "gender"] = data["gender"].fillna("unknown")

In [10]:
education_mapping = {'Less than Secondary': 0, 'Secondary': 1, "Bachelor's": 2, "Master's": 3, "Doctorate": 4}
data["LoE_DI"] = data["LoE_DI"].map(education_mapping)

gender_mapping = {"m": 0, "f": 1, "o": 2, "unknown": 3}
data["gender"] = data["gender"].map(gender_mapping)

from sklearn.preprocessing import LabelEncoder

le_course = LabelEncoder()
data["course_id_enc"] = le_course.fit_transform(data["course_id"])

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical features
le_gender = LabelEncoder()
le_edu = LabelEncoder()
le_field = LabelEncoder()

data["gender_enc"] = le_gender.fit_transform(data["gender"])
data["LoE_DI_enc"] = le_edu.fit_transform(data["LoE_DI"])
data["field_of_study_enc"] = le_field.fit_transform(data["field_of_study"])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# ----------------------------
# Step 1: Encode `course_id` if not already done
# ----------------------------
# Check if 'course_id_enc' exists; if not, create it
if 'course_id_enc' not in data.columns:
    le_course = LabelEncoder()
    data['course_id_enc'] = le_course.fit_transform(data['course_id'])

X = data[['LoE_DI_enc', 'YoB', 'gender_enc', 'field_of_study_enc']]
y = data['course_id_enc']  # Use the encoded course IDs

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# LightGBM - fast and high performance
import lightgbm as lgb

# Create the model
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.1,
    n_estimators=300,
    random_state=42
)

# Train
lgb_model.fit(X_train, y_train)

# Evaluate
y_pred = lgb_model.predict(X_test)
print(f"LightGBM Accuracy: {accuracy_score(y_test, y_pred):.4f}")

ModuleNotFoundError: No module named 'lightgbm'